In [1]:
import requests
from bs4 import BeautifulSoup
import logging
import json
from time import sleep

In [2]:
BASE_URL           = "https://sandiego.nextrequest.com"
REQUESTS_TABLE_URL = "https://sandiego.nextrequest.com/requests?requests_smart_listing[page]="
REQUEST_PAGE_URL   = "https://sandiego.nextrequest.com/requests"
PAGES_COUNT        = 682 # going past the last page just forwards back to the last one
SUCCESS_CODE       = 200

## Test Code Below

This pull was last run at 2p.m. Monday November 25, 2019. Around 17,000 public records request IDs acquired.

In [7]:
pra_ids = []
for i in range(1,PAGES_COUNT):
    page_request = requests.get(REQUESTS_TABLE_URL + str(i))
    
    if page_request.status_code != SUCCESS_CODE:
        logging.warning("FAILED CODE " + str(page_request.status_code) + "FOR PAGE " + REQUESTS_TABLE_URL + str(i))
        
    if (i % 100 == 0):
        print(str(i) + " PAGES COMPLETED")
        #logging.info()
        
    page_text = BeautifulSoup(page_request.text)

    request_table = page_text.find_all("table", class_="request_table")[0]
    
    for row in request_table.tbody.findAll('tr'):
        pra_ids.append(row.findAll('td')[0].a.get('href'))

100 PAGES COMPLETED
200 PAGES COMPLETED
300 PAGES COMPLETED
400 PAGES COMPLETED
500 PAGES COMPLETED
600 PAGES COMPLETED


In [13]:
page_request = requests.get(REQUESTS_TABLE_URL + str(680))

if page_request.status_code != SUCCESS_CODE:
    logging.warning("FAILED CODE " + str(page_request.status_code) + "FOR PAGE " + REQUESTS_TABLE_URL + str(i))
    
if (i % 100 == 0):
    print(str(i) + " PAGES COMPLETED")
    #logging.info()
    
page_text = BeautifulSoup(page_request.text)

request_table = page_text.find_all("table", class_="request_table")[0]

for row in request_table.tbody.findAll('tr'):
    pra_ids.append(row.findAll('td')[0].a.get('href'))

In [14]:
len(pra_ids)

17006

In [17]:
with open('pra_href_scrape.txt', 'w') as outfile:
    json.dump(pra_ids, outfile)

## Scrape Request Page

In [11]:
# READ IN THE REQUEST IDs
with open('pra_href_scrape.txt') as json_file:
    pra_ids = json.load(json_file)

In [9]:
r_page = BeautifulSoup(requests.get(REQUEST_PAGE_URL + '/19-5607').text)

In [23]:
def clean_empty_space(string):
    return " ".join(string.split())

In [27]:
def get_rpage_data(pra_link):
    r_data_i = {}
    
    r_page = BeautifulSoup(requests.get(BASE_URL + pra_link).text)
    
    
    r_id = pra_link.split("/")[-1]
    
    r_text = r_page.find("div", {"id": "request-text"}).p.get_text() # get request text

    r_date = r_page.findAll("p", {"class": "request_date"})[0].get_text() # get the submission date
    r_date = r_date.split()
    r_channel = r_date[-1]
    r_date = " ".join(r_date[:3])
    
    r_department = r_page.findAll("p", {"class": "current-department"})[0].get_text() # get the respective department
    r_department = clean_empty_space(r_department)
    r_contact = ' '.join(r_page.findAll("div", {"class": "staff-details"})[0].text.split("Contact",1)[1].split()) # get the request contact
    
    r_events = r_page.findAll("div", {"class": "generic-event"}) # get each public event
    
    for event in r_events:
        event_title = event.findAll("div", {"class": "event-title"})[0].text
        event_text  = event.findAll("div", {"class": "event-item"})[0].text
        event_title = ' '.join(event_title.split())
        event_text  = ' '.join(event_text.split())
        if (event_title == "Request Closed"):
            r_data_i["Close_Date"] = event_title + " " + " ".join(event.findAll("div")[-1].get_text().split()[:3]).replace(",","")
        if(~(event_title in r_data_i.keys())):
            r_data_i[event_title] = event_text
        else:
            r_data_i[event_title] = r_data_i[event_title] + "; " + event_text
    
    row = [r_id, r_date, r_channel, r_department, r_contact, r_data_i]
    sleep(1)
    return row

In [28]:
[get_rpage_data(link) for link in pra_ids[:10]]

[['19-5614',
  'November 21, 2019',
  'web',
  'Fire-Rescue',
  'Angela Laurita',
  {'Request Published Public': '',
   'Request Closed Public': 'Duplicate request This request was already entered under a different number.',
   'Department Assignment Public': 'Fire-Rescue',
   'Request Opened Public': 'Request received via web'}],
 ['19-5607',
  'November 21, 2019',
  'web',
  'Public Records Administration',
  'Angela Laurita',
  {'Request Published Public': '',
   'Request Closed Public': 'No responsive documents The City of San Diego has no responsive documents.',
   'Document(s) Released Public': 'https://www.sandiego.gov/sustainability/energy-and-water-efficiency/programs-projects/smart-city',
   'Department Assignment Public': 'Public Records Administration',
   'Request Opened Public': 'Request received via web'}],
 ['19-5605',
  'November 20, 2019',
  'email',
  'Code Enforcement',
  'Ginger Rodriguez',
  {'Request Published Public': '',
   'Request Closed Public': '02. Release